In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm')
import inflect
p = inflect.engine()

In [ ]:
PORTUGUESE_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Todos os {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cada um dos {a_pl} pertence à classe de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sem exceção, todos os {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'A categoria dos {a_pl} inclui {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'A totalidade dos {a_pl} corresponde a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cada {a_sg} é {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Se algo é {a_sg_art}, então é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Tudo o que for {a_sg_art} é também {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'A classe {a_pl} está contida na classe {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Qualquer {a_sg} qualifica-se como {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Não existe nenhum {a_sg} que não seja {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Ser {a_sg_art} implica ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Todo e qualquer {a_sg} é {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Os {a_pl} são universalmente {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Cada membro de {a_pl} é também membro de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Alguns {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existem {a_pl} que são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Uma porção dos {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Parte dos {a_pl} pertence a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Entre os {a_pl}, alguns são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existe pelo menos um {a_sg} que é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Certos {a_pl} resultam ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'É possível encontrar {a_pl} que sejam {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Pelo menos uma instância de {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Há casos em que {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Nem tudo, mas alguns {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Podemos apontar {a_pl} que são também {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Algumas instâncias de {a_pl} caem sob {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Vários {a_pl} são identificados como {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ocorre que alguns {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Nenhum dos {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Não existem {a_pl} que sejam {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': '{a_pl} e {b_pl} não se sobrepõem.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'A classe dos {a_pl} não contém {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Absolutamente nenhum {a_pl} pertence a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Não existe {a_sg} que seja {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Nada que seja {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'É falso que algum {a_sg} seja {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'A classe de {a_pl} é disjunta da classe {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sob nenhuma circunstância um {a_sg} é {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Fica excluído que os {a_pl} sejam {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Se é {a_sg_art}, não pode ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Não se pode achar um {a_sg} que seja {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Nenhum membro de {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'A interseção entre {a_pl} e {b_pl} é vazia.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Alguns {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Há {a_pl} que não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Uma parcela dos {a_pl} não pertence a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nem todos os {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Entre os {a_pl}, alguns não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nem todo {a_sg} é {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Certos {a_pl} carecem da qualidade de ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existe pelo menos um {a_sg} que não é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Pelo menos algumas instâncias de {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Não é o caso de que cada {a_sg} seja {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Parte dos {a_pl} está fora de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Podemos encontrar {a_pl} que não resultam ser {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nem tudo o que é {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Alguns dos {a_pl} não são membros de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Existem exemplares de {a_pl} que não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Portanto, {conclusion}.',
        'Segue-se que {conclusion}.',
        'Consequentemente, {conclusion}.',
        'Logo, {conclusion}.',
        'Em resumo: {conclusion}.',
        'De tal modo que {conclusion}.',
        'Por conseguinte, {conclusion}.',
        'Assim sendo, {conclusion}.',
        'Conclui-se que {conclusion}.',
        'Deste modo, {conclusion}.'
    ]
}

In [ ]:
PORTUGUESE_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Todo {d_sg} que é {a_sg_art} também é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Se {d_sg_art} for {a_sg_art}, então é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Todos os {d_pl} que são {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Sem exceção, todo {d_sg} com a propriedade {a_sg} é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Para cada {d_sg} que é {a_sg_art}, é verdade que é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Qualquer {d_sg} que se qualifique como {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Se um objeto for {d_sg_art} e {a_sg_art}, necessariamente será {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'A totalidade dos {d_pl} que são {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Dentro de {d_pl}, todos os {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Para qualquer {d_sg}, se este for {a_sg_art}, então é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Tudo o que for simultaneamente {d_sg_art} e {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Cada membro de {d_pl} que pertence a {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Qualquer um dos {d_pl} que seja {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Não há nenhum {d_sg} que sendo {a_sg_art} não seja {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Todo exemplar de {d_sg} que for {a_sg} será {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'some': [
            {'tpl': 'Existe pelo menos um {d_sg} que é tanto {a_sg} quanto {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Alguns {d_pl} que são {a_pl} são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Uma parte dos {d_pl} que são {a_pl} pertence a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Há {d_pl} que são {a_pl} e também {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ao menos um {d_sg} que é {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Certos {d_pl} resultam ser tanto {a_pl} quanto {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'É possível encontrar um {d_sg} que seja {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ocorre que algum {d_sg} é simultaneamente {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Uma porção de {d_pl} apresenta qualidades de {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Há pelo menos um exemplo de {d_sg} que é {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Podemos achar {d_pl} que cumprem ser {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Algum {d_sg} é ao mesmo tempo {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Dentro da classe {d_pl}, existem {a_pl} que são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Vários {d_pl} partilham as características de {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Algumas instâncias de {d_sg} são {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'no': [
            {'tpl': 'Nenhum dos {d_pl} que são {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Não existe {d_sg} que seja {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'A classe dos {d_pl} que são {a_pl} não contém {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'É impossível que um {d_sg} seja {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Nenhum {d_sg} que é {a_sg} é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Não há instâncias de {d_pl} que sejam simultaneamente {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Nenhum {d_sg} com a propriedade {a_sg} possui a propriedade {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Dentro de {d_pl}, nenhum membro de {a_pl} é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Fica excluído que os {d_pl} que sejam {a_pl} sejam também {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'É falso que algum {d_sg} seja a um só tempo {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Nenhum dos membros de {d_pl} pertence a {a_pl} e {b_pl} ao mesmo tempo.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Se for {d_sg_art} e {a_sg_art}, então não pode ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Não há forma de que um {d_sg} seja {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'A categoria {d_pl} não admite a combinação de {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Qualquer {d_sg} que seja {a_sg} não é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'some_not': [
            {'tpl': 'Nem todo {d_sg} que é {a_sg_art} é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Há pelo menos um {d_sg} que é {a_sg} mas não é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Alguns {d_pl} que são {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Uma parte dos {d_pl} com a propriedade {a_pl} não pertence a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Existem {d_pl} que são {a_pl} e ainda assim não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Certos {d_pl} que pertencem a {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Existe ao menos um {d_sg} que, sendo {a_sg_art}, não é {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Não é verdade que cada {d_sg} que seja {a_sg_art} seja também {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Podemos achar um {d_sg} que é {a_sg} mas não é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Dentro da classe {d_pl}, alguns {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Nem tudo o que é simultaneamente {d_sg_art} e {a_sg_art} resulta ser {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Algum {d_sg} que é {a_sg} não é {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'É possível que um {d_sg} seja {a_sg} e no entanto não seja {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Vários membros de {d_pl} que são {a_pl} não são {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Uma parcela de {d_pl} é {a_pl} mas não é {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
    }
}

In [ ]:
def get_singular_and_plural(term):
    term = term.strip()
    sg_attempt = p.singular_noun(term)
    if sg_attempt is not False:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        pl = p.plural_noun(term)
    if sg is False:
        sg = term
    if not pl:
        pl = term
    return sg, pl

In [ ]:
def get_article(term):
    term = term.strip()
    return p.an(term)

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    # 4th-variable patterns first
    match = re.match(r'All (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*)\.', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    # basic forms
    match = re.match(r'All (.*) are (.*)\.', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*)\.', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*)\.', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*)\.', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    sentences = [s.strip() for s in simple_syllogism.split('.') if s.strip()]
    sentences = [s + '.' for s in sentences]
    premises_simple = sentences[:-1]
    conclusion_simple = sentences[-1]
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f" Skipping item ID {item.get('id', 'N/A')} due to parsing error: {e}")
            continue
    return complex_dataset

In [ ]:
# Example data and run (sanity check)
sample_data = [
    {'id': 'pt1', 'syllogism': 'All gatos são animais. Alguns animais de estimação são gatos. Alguns animais de estimação são animais.', 'validity': True},
    {'id': 'pt2', 'syllogism': 'Nenhum carro é pássaro. Alguns veículos são carros. Alguns veículos não são pássaros.', 'validity': True}
]
complex_syllogism_data = process_dataset(sample_data, PORTUGUESE_COMPLEX_PATTERNS, PORTUGUESE_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print('
--- Converted Portuguese Complex Syllogisms ---')
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))